In [52]:
import tensorflow as tf
import numpy as np
from sklearn.preprocessing import OneHotEncoder
tf.reset_default_graph()
sess = tf.InteractiveSession()

In [53]:
from collections import namedtuple
group_t = namedtuple('group_t', ['id', 'inds'])

In [54]:
groups = [group_t(0, [0, 1, 2]), group_t(3, [3, 4]), group_t(5, [5])]

In [55]:
enc = OneHotEncoder(n_values=6)
batch = 10
labels = tf.constant(enc.fit_transform(np.random.randint(low=0, high=6, size=(batch, 1))).toarray(), dtype=tf.float32)
a = np.random.rand(batch, 6)
probs = tf.constant(a / np.sum(a, axis=1, keepdims=True), dtype=tf.float32)

In [56]:
sess.run(labels)

array([[0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 1., 0.],
       [0., 0., 1., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0.],
       [0., 1., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 1.]], dtype=float32)

In [57]:
sess.run(probs)

array([[0.28148592, 0.12334423, 0.27818066, 0.05824978, 0.17000529,
        0.08873413],
       [0.15645288, 0.01986464, 0.21464635, 0.14941627, 0.23740195,
        0.22221792],
       [0.02086289, 0.14950497, 0.12770204, 0.21368167, 0.30405772,
        0.18419072],
       [0.13329518, 0.08393449, 0.24693975, 0.16067754, 0.22760928,
        0.14754376],
       [0.2603274 , 0.19912387, 0.01489106, 0.16402319, 0.30266422,
        0.05897028],
       [0.31034127, 0.03975685, 0.02884524, 0.25728238, 0.08843639,
        0.2753379 ],
       [0.01865423, 0.13742082, 0.2524908 , 0.05165657, 0.16414306,
        0.37563452],
       [0.05662037, 0.2012304 , 0.20134567, 0.1700586 , 0.09199238,
        0.27875257],
       [0.17539476, 0.17781967, 0.17155196, 0.21464176, 0.2035678 ,
        0.05702404],
       [0.20238094, 0.1111108 , 0.18352216, 0.40888548, 0.02075121,
        0.07334941]], dtype=float32)

In [58]:
def upper_feature(probs, groups):
    res = []
    for group in sorted(groups, key=lambda x : x.id):
        res.append(tf.reduce_sum(tf.gather(probs, group.inds, axis=-1)))
    return tf.stack(res, axis=-1)

In [59]:
v = upper_feature(probs, groups)
sess.run(v)

array([4.5790424, 3.6592023, 1.7617552], dtype=float32)

In [72]:
def lower_label(labels, groups):
    indices = []
    values = []
    for group in sorted(groups, key=lambda x : x.id):
        for ind in sorted(group.inds):
            if ind == group.id:
                for i in group.inds:
                    indices.append([i, ind])
                    values.append(1.0 / len(group.inds))
            else:
                indices.append([ind, ind])
                values.append(1.0)
    z = sorted(zip(indices, values), key=lambda x : x[0])
    indices, values = zip(*z)
    sp = tf.SparseTensor(indices=indices, values=np.float32(values), dense_shape=(labels.shape[-1], labels.shape[-1]))
    return tf.transpose(tf.sparse_tensor_dense_matmul(sp, tf.transpose(labels)))

In [73]:
lower_labels = lower_label(labels, groups)

In [74]:
sess.run(lower_labels)

array([[0.        , 0.        , 1.        , 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , 0.        , 0.        ,
        1.        ],
       [0.        , 0.        , 0.        , 0.        , 0.        ,
        1.        ],
       [0.        , 0.        , 0.        , 0.        , 1.        ,
        0.        ],
       [0.        , 0.        , 1.        , 0.        , 0.        ,
        0.        ],
       [0.33333334, 0.33333334, 0.33333334, 0.        , 0.        ,
        0.        ],
       [0.33333334, 0.33333334, 0.33333334, 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , 0.        , 1.        ,
        0.        ],
       [0.        , 1.        , 0.        , 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , 0.        , 0.        ,
        1.        ]], dtype=float32)